In [2]:
import random
import base64
from io import BytesIO
from PIL import Image
import numpy as np
import cv2
import requests
import json
from glob import glob
from multiprocessing import Pool
from time import time
from tqdm import tqdm
import os
from pprint import pprint
import pickle as cpk
from concurrent import futures
import shutil

import cv2
import matplotlib.pyplot as plt

In [10]:
# 开关rtmp流的推送与拉取
def rtmp_request(user_id, action, rtmp_url):
    url = "http://yyx.play.cn:11802/sm/bizcmd/v2/livectrl"
    headers = {"Content-Type": 'application/json;charset=UTF-8'}
    params = {
    "tenantId": "cybercloud",
    "userId": user_id,
    "action": action,
    "uploadType": "",
    "uploadUrl": rtmp_url,
    "exParams": {}
    }

    response = requests.post(url, headers=headers, data=json.dumps(params))
    print(response.text)


user_id = "1741004057"
action = 'stop' # start; stop
rtmp_url = "rtmp://218.104.48.105:1935/live/test"

rtmp_request(user_id=user_id, 
             action=action,
             rtmp_url=rtmp_url)

{"Description":"unknown error","ResultCode":3473415,"StreamSessionID":"","TenantID":"cybercloud","UserID":"1741004057"}



In [7]:
# 获取流对象
cap = cv2.VideoCapture(rtmp_url)
fps = cap.get(cv2.CAP_PROP_FPS)

store_dir = './data/quanhuang'
if not os.path.exists(store_dir):
    os.makedirs(store_dir)

In [9]:
ret = True

while ret:
    img_name = str(int(time()*1000000)) + '.jpg'
    path = os.path.join(store_dir, img_name)
    ret, frame = cap.read()
    cv2.imwrite(path, frame)
print('stream is stop...')

cap.release()

stream is stop...


In [1]:
# paddle ocr client

import random
import base64
from io import BytesIO
from PIL import Image
import numpy as np
import cv2
import requests
import json
from glob import glob
from multiprocessing import Pool
from time import time
from tqdm import tqdm
import os
from pprint import pprint
import pickle as cpk


def request_api(url, header, params):
    response = requests.post(url, headers=header, data=json.dumps(params))
    response = response.json()
    return response

def base64_to_image(base64_str, channel=3):
    img_data = base64.b64decode(base64_str)
    nparr = np.frombuffer(img_data, np.uint8)
    if channel == 1:
        img = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
    if channel == 3:
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img

def image_to_base64(img_path):
    with open(img_path, 'rb') as f:
        base64_bytes = base64.b64encode(f.read())
    return base64_bytes.decode()

def array_to_base64(arr):
    retval, buffer = cv2.imencode('.jpg', arr)
    arr_str = base64.b64encode(buffer)
    arr_str = arr_str.decode()
    return arr_str

def base64_to_array(base64_str):
    img_data = base64.b64decode(base64_str)
    arr = cpk.loads(img_data)
    return arr

def pil_to_byte(img):
    buffered = BytesIO()
    img.save(buffered, format=img.format)
    return buffered.getvalue()

def data_process(img):
    base64_bytes = array_to_base64(img)
    param = {"image": base64_bytes}
    return param

def get_request_result(data, url, headers):
    param = data_process(data)
    rst = request_api(url, headers, param)
    return rst

def get_request_by_eureka(data, app_name, service):
    params = data_process(data)
    response = eureka_client.do_service(
        app_name = app_name,
        service = service,
        method = 'POST',
        headers = headers,
        return_type = 'json',
        data = json.dumps(params).encode(),
        timeout=60
    )
    return response

In [ ]:
url = "http://47.98.153.185:55888/general/ocr"
headers = {"Content-Type": 'application/json;charset=UTF-8'}
img_paths = glob('./data/quanhuang/*.jpg')
for imgp in tqdm(img_paths[4260::5]):
    img_name = os.path.basename(imgp)
    txt_name = img_name.replace('jpg', 'txt')
    txt_store_path = os.path.join('./data/quanhuang', txt_name)
    img = cv2.imread(imgp)
    if img is None:
        continue
    base64_bytes = array_to_base64(img)
    params = {"image": base64_bytes}
    response = requests.post(url, headers=headers, data=json.dumps(params))
    try:
        response = response.json()
        content = response.get('content', 'None')
        with open(txt_store_path, 'w') as f:
            f.write('\n'.join(content))
    except:
        continue


  6%|▌         | 131/2115 [05:52<1:13:28,  2.22s/it]